In [1]:
%run uvms_forward_kinematics.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4
Task Space Box:
{'x_min': -0.28692105492352493, 'x_max': 0.054610904328225296, 'y_min': -0.038530132874644796, 'y_max': 0.20243427434910413, 'z_min': 0.01477899230715364, 'z_max': 0.3100041333006748}


In [2]:
J = dIFF_KinJ4
J_plus = cs.pinv(J)
I = cs.SX.eye(J.size2())

J_ned = cs.SX.zeros((10, 10))
J_ned[:6,:6] = J_uv
J_ned[6:,6:] = cs.SX.eye(4)

J_ned_inv = cs.inv(J_ned)

In [3]:
# joint limit secondary task
# stay away from the joint limits 

sigma_wq_i = 0
for i in range(uvms_ss.total_dof):
    q_i_bar = (uvms_ss.ul[i] + uvms_ss.ll[i])/2
    wq_i = ((uvms_ss.n[i] - q_i_bar)/(uvms_ss.ul[i] - uvms_ss.ll[i]))**2
    sigma_wq_i = sigma_wq_i + wq_i
    
    
wq =  (1/(2*uvms_ss.total_dof))*sigma_wq_i

J_wq = cs.jacobian(wq, uvms_ss.n)

qdot_sec = cs.diag(uvms_ss.k0)@J_wq.T

In [4]:
# task projection
IK_v = J_ned_inv@(J_plus@uvms_ss.des_v + (I - J_plus@J)@qdot_sec)
IK_v_func = cs.Function('d_iK', [uvms_ss.des_v, uvms_ss.n, uvms_ss.ul, uvms_ss.ll, uvms_ss.k0, base_ss.base_T], [IK_v])
# IK_v_func.save('diff_iK.casadi')

In [1]:
# differential inverse kinematics
# IK_v

In [6]:
IK_v_func([0,0,0, 0,0,0], [0,0,0, 0.01,0.01,0.01 ,1,1,1,1], [10, 10, 10, cs.pi,cs.pi,cs.pi, 2*cs.pi, 2*cs.pi, 2*cs.pi, 2*cs.pi], [-10, -10, -10, -cs.pi,-cs.pi,-cs.pi, 0,0,0,0], [1,1,1, 1,1,1, 1,1,1,1], alpha.base_T0)

DM([0.000692876, 0.000732804, -0.000779058, 3.15914e-05, 1.98439e-05, 2.45993e-05, -0.00532236, -0.00534529, -0.0053826, -0.00534664])